# Sentiment classification comparison between RoBERTa and ChatGPT models.<br>

This notebook evaluates the performance of the fine-tuned RoBERTa model against<br>
OpenAI's **ChatGPT** (specifically, gpt-3.5-turbo) on a sampled subset of the<br>
Amazon reviews. It includes a series of visualizations and a summary of key metrics.<br>

In [ ]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoConfig
)
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    precision_recall_fscore_support,
)
from openai import OpenAI
from dotenv import load_dotenv

plt.style.use('default')
sns.set_palette("husl")

## Final Sentiment Prediction and Export

In [ ]:
df = pd.read_csv(
    "../data/processed/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_cleaned.csv"
)
df['combined_review'] = df['title'] + " " + df['text']
MODEL_DIR = (
    r"C:\Users\nicol\Desktop\Ironhack\week6\Project_NLP\models\cardiffnlptwitter-roberta-base-sentiment-latest"
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
config = AutoConfig.from_pretrained(MODEL_DIR)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR, config=config)

In [ ]:
predictions_roberta = []
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Batch processing for efficiency on the full dataset

In [ ]:
BATCH_SIZE = 32
for batch_idx in range(0, len(df), BATCH_SIZE):
    batch = df['combined_review'].iloc[batch_idx:batch_idx + BATCH_SIZE].tolist()

    # Ensure all reviews in the batch are strings
    batch = [str(review) if not pd.isna(review) else "" for review in batch]
    inputs = tokenizer(
        batch, return_tensors="pt", padding=True, truncation=True, max_length=256
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class_ids = logits.argmax(dim=-1).cpu().numpy()
    predicted_labels = [
        model.config.id2label.get(id, str(id)) for id in predicted_class_ids
    ]
    predictions_roberta.extend(predicted_labels)

In [ ]:
df['predicted_sentiment_roberta'] = predictions_roberta
df['predicted_sentiment_roberta'] = df['predicted_sentiment_roberta'].str.lower()

Save the updated DataFrame to a new CSV file

In [ ]:
OUTPUT_FILE_PATH = "../data/processed/Datafiniti_with_sentiments.csv"
df.to_csv(OUTPUT_FILE_PATH, index=False)

## Visualization & Scores

In [ ]:
load_dotenv()

In [ ]:
class SentimentAnalyzer:
    """Class for analyzing and comparing sentiment models."""
    def __init__(self, data_df):
        self.df = data_df.copy()
        self.client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
    def create_subset(self, n_per_class=20):
        """Create balanced subset for comparison."""
        subset_list = []
        for sentiment in ['positive', 'negative', 'neutral']:
            class_data = self.df[self.df['star_sentiment'].str.lower() == sentiment]
            if len(class_data) >= n_per_class:
                subset_list.append(class_data.sample(n=n_per_class, random_state=42))
        return pd.concat(subset_list, ignore_index=True)
    def get_chatgpt_prediction(self, text):
        """Get ChatGPT prediction for sentiment."""
        try:
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{
                    "role": "system",
                    "content": ("You are a sentiment classifier. Respond with EXACTLY "
                               "one word: Positive, Negative, or Neutral. No other text.")
                }, {
                    "role": "user",
                    "content": f"Classify this sentiment: {text[:500]}"
                }],
                max_tokens=1,
                temperature=0
            )
            result = response.choices[0].message.content.strip()
            if result in ['Positive', 'Negative', 'Neutral']:
                return result
            return 'Neutral'
        except Exception:
            return 'Neutral'
    def analyze_chatgpt_subset(self, subset_df):
        """Analyze subset with ChatGPT predictions."""
        chatgpt_preds = []
        for _, row in subset_df.iterrows():
            text = str(row.get('combined_review', row.get('text', '')))
            pred = self.get_chatgpt_prediction(text)
            chatgpt_preds.append(pred)
            if len(chatgpt_preds) % 10 == 0:
                print(f"Processed {len(chatgpt_preds)}/{len(subset_df)}")
            time.sleep(0.5)
        subset_df = subset_df.copy()
        subset_df['chatgpt_prediction'] = chatgpt_preds
        return subset_df
    def plot_roberta_analysis(self):
        """Plot RoBERTa analysis results."""
        df_clean = self.df.dropna(subset=['star_sentiment', 'predicted_sentiment_roberta'])
        df_clean['star_sentiment'] = df_clean['star_sentiment'].str.lower()
        df_clean['predicted_sentiment_roberta'] = (
            df_clean['predicted_sentiment_roberta'].str.lower()
        )
        y_true = df_clean['star_sentiment']
        y_pred = df_clean['predicted_sentiment_roberta']
        labels = ['negative', 'neutral', 'positive']
        _, axes = plt.subplots(2, 2, figsize=(15, 10))
        plt.suptitle('RoBERTa vs True Labels Analysis', fontsize=14, fontweight='bold')

        # Confusion Matrix
        cm = confusion_matrix(y_true, y_pred, labels=labels)
        cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues',
                   xticklabels=labels, yticklabels=labels, ax=axes[0, 0])
        axes[0, 0].set_title('Confusion Matrix')

        # Accuracy by class
        precision, recall, f1, _ = precision_recall_fscore_support(
            y_true, y_pred, labels=labels
        )
        x = np.arange(len(labels))
        width = 0.25
        axes[0, 1].bar(x - width, precision, width, label='Precision', alpha=0.8)
        axes[0, 1].bar(x, recall, width, label='Recall', alpha=0.8)
        axes[0, 1].bar(x + width, f1, width, label='F1-Score', alpha=0.8)
        axes[0, 1].set_xticks(x)
        axes[0, 1].set_xticklabels(labels)
        axes[0, 1].legend()
        axes[0, 1].set_title('Performance by Class')

        # Distribution comparison
        true_counts = y_true.value_counts()
        pred_counts = y_pred.value_counts()
        x = np.arange(len(labels))
        axes[1, 0].bar(
            x - 0.2, [true_counts.get(l, 0) for l in labels], 0.4,
            label='True', alpha=0.8
        )
        axes[1, 0].bar(
            x + 0.2, [pred_counts.get(l, 0) for l in labels], 0.4,
            label='Predicted', alpha=0.8
        )
        axes[1, 0].set_xticks(x)
        axes[1, 0].set_xticklabels(labels)
        axes[1, 0].legend()
        axes[1, 0].set_title('Distribution Comparison')

        # Overall accuracy
        accuracy = accuracy_score(y_true, y_pred)
        axes[1, 1].bar(['RoBERTa'], [accuracy], color='green', alpha=0.7)
        axes[1, 1].set_ylabel('Accuracy')
        axes[1, 1].set_title(f'Overall Accuracy: {accuracy:.3f}')
        axes[1, 1].set_ylim(0, 1)
        plt.tight_layout()
        plt.show()
        print(f"RoBERTa Accuracy: {accuracy:.3f}")
        print(classification_report(y_true, y_pred, target_names=labels))
        return accuracy
    def compare_models(self, comparison_subset):
        """Compare RoBERTa and ChatGPT models."""
        df_clean = comparison_subset.dropna()
        df_clean['star_sentiment'] = df_clean['star_sentiment'].str.lower()
        df_clean['predicted_sentiment_roberta'] = (
            df_clean['predicted_sentiment_roberta'].str.lower()
        )
        df_clean['chatgpt_prediction'] = df_clean['chatgpt_prediction'].str.lower()
        y_true = df_clean['star_sentiment']
        y_roberta = df_clean['predicted_sentiment_roberta']
        y_chatgpt = df_clean['chatgpt_prediction']
        roberta_accuracy = accuracy_score(y_true, y_roberta)
        chatgpt_accuracy = accuracy_score(y_true, y_chatgpt)
        _, axes = plt.subplots(1, 3, figsize=(18, 5))

        # Accuracy comparison
        models = ['ChatGPT', 'RoBERTa']
        accuracies = [chatgpt_accuracy, roberta_accuracy]
        axes[0].bar(models, accuracies, color=['#FF6B6B', '#4ECDC4'], alpha=0.8)
        axes[0].set_ylabel('Accuracy')
        axes[0].set_title('Model Comparison')
        axes[0].set_ylim(0, 1)
        for idx, v in enumerate(accuracies):
            axes[0].text(idx, v + 0.01, f'{v:.3f}', ha='center')

        # ChatGPT confusion matrix
        labels = ['positive', 'negative', 'neutral']
        cm_chatgpt = confusion_matrix(y_true, y_chatgpt, labels=labels)
        sns.heatmap(cm_chatgpt, annot=True, fmt='d', cmap='Reds',
                   xticklabels=labels, yticklabels=labels, ax=axes[1])
        axes[1].set_title('ChatGPT Confusion Matrix')

        # RoBERTa confusion matrix
        cm_roberta = confusion_matrix(y_true, y_roberta, labels=labels)
        sns.heatmap(cm_roberta, annot=True, fmt='d', cmap='Blues',
                   xticklabels=labels, yticklabels=labels, ax=axes[2])
        axes[2].set_title('RoBERTa Confusion Matrix')
        plt.tight_layout()
        plt.show()
        print(f"ChatGPT Accuracy: {chatgpt_accuracy:.3f}")
        print(f"RoBERTa Accuracy: {roberta_accuracy:.3f}")
        print(f"Agreement Rate: {(y_chatgpt == y_roberta).mean():.3f}")
        return chatgpt_accuracy, roberta_accuracy

In [ ]:
df = pd.read_csv("../data/processed/Datafiniti_with_sentiments.csv")
analyzer = SentimentAnalyzer(df)

In [ ]:
roberta_accuracy = analyzer.plot_roberta_analysis()

In [ ]:
subset = analyzer.create_subset(n_per_class=20)
subset_with_chatgpt = analyzer.analyze_chatgpt_subset(subset)
chatgpt_acc, roberta_acc = analyzer.compare_models(subset_with_chatgpt)